# Import 

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import joblib
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, 
                             roc_curve, roc_auc_score, precision_recall_curve, classification_report, auc,confusion_matrix)

## Coloque os seus dados já tratados aqui
data = pd.read_csv('dados_tratados.csv', index_col='Unnamed: 0')
data

# Balanceando os target

In [ ]:
# Separando os atributos e os target
X = data.drop('target', axis=1)
y = data['target']

# Criando o objeto de oversampling
ros = RandomOverSampler(random_state=42)

# Aplicando o oversampling para balancear as classes
X_res, y_res = ros.fit_resample(X, y)

# Separando em treino e teste

In [ ]:
# Separando entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X_res, y_res,
    test_size=0.3,         
    random_state=42,       
    stratify=y_res              
)

# Aplicando GridSearch

In [ ]:
# Definir o modelo RandomForest
rf = RandomForestClassifier(random_state=42)

# Definir a grade de parâmetros (grid) para o RandomForest
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Configurar o GridSearchCV
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=10,                
    scoring='accuracy',  
    n_jobs=-1,           
    verbose=1            
)

# Aplicar o GridSearchCV aos dados de treino
grid_search.fit(X_train, y_train)

# Exibir os melhores parâmetros e a melhor acurácia encontrada na validação
print("Melhores parâmetros encontrados:", grid_search.best_params_)
print("Melhor acurácia na validação cruzada:", grid_search.best_score_)

# Avaliar o modelo selecionado no conjunto de teste
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("\nAcurácia no conjunto de teste:", test_accuracy)
print("\nRelatório de Classificação no conjunto de teste:\n", classification_report(y_test, y_pred))

# Gerando gráfico de Análise

## Matriz de confusão

In [ ]:
# Calcula a matriz de confusão
cm = confusion_matrix(y_test, y_pred)

# Define os rótulos customizados para as classes:
# 0 --> "não fechou o gap"
# 1 --> "fechou o gap"
classes = ['Não fechou o GAP', 'Fechou o GAP']

# Plota a matriz de confusão utilizando seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=classes, yticklabels=classes)
plt.xlabel('Valor Predito')
plt.ylabel('Valor Verdadeiro')
plt.title('Matriz de Confusão')
plt.show()

## Curva ROC

In [ ]:
y_scores = best_model.predict_proba(X_test)
y_scores = [x for _,x in y_scores]
# Calcula a curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_scores)

# Calcula a AUC (Area Under the Curve)
roc_auc = auc(fpr, tpr)

# Plot da curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2,
         label='Curva ROC (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--',
         label='Linha de referência (AUC = 0.5)')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de Falsos Positivos')
plt.ylabel('Taxa de Verdadeiros Positivos')
plt.title('Curva ROC')
plt.legend(loc='lower right')
plt.show()

## Analisando distribuição da acurácia, precision e recall do modelo

In [ ]:
list_acuracia = []
list_precision = []
list_recall = []
for x in range(1,101):
    rf = best_model
    X_train, X_test, y_train, y_test = train_test_split(
    X_res, y_res,
    test_size=0.3,         
    random_state=x,       
    stratify=y_res              
    )
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    test_precision = precision_score(y_test, y_pred)
    test_recall = recall_score(y_test, y_pred)
    list_acuracia.append(test_accuracy)
    list_precision.append(test_precision)
    list_recall.append(test_recall)
# Ploando o gráfico
plt.figure(figsize = (16,9))
plt.hist(list_acuracia, bins = [x/100 for x in range(50,101,1)], edgecolor = 'black', label = 'Acurácia', color = 'blue')
plt.hist(list_precision, bins = [x/100 for x in range(50,101,1)], edgecolor = 'black',color = 'orange', label = 'Precision', alpha = .5)
plt.hist(list_acuracia, bins = [x/100 for x in range(50,101,1)], edgecolor = 'black', color = 'green', label = 'Recall', alpha = .5)
plt.title('Distribuição da acurácia do meu modelo')
plt.ylabel('Frequência')
plt.xlabel('Acurácia')
plt.xticks([x/100 for x in range(50,101,1)], rotation = 'vertical')
plt.legend()
plt.grid()
plt.show()

## Gráfico de Métricas vs Threshold

In [ ]:
# Obter as probabilidades preditas para a classe positiva (1)
y_prob = best_model.predict_proba(X_test)[:, 1]

# Calcular as métricas para diferentes thresholds
thresholds = np.linspace(0, 1, 100)
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []

for thresh in thresholds:
    # Classifica com base no threshold atual
    y_pred_thresh = (y_prob >= thresh).astype(int)
    accuracy_list.append(accuracy_score(y_test, y_pred_thresh))
    precision_list.append(precision_score(y_test, y_pred_thresh))
    recall_list.append(recall_score(y_test, y_pred_thresh))
    f1_list.append(f1_score(y_test, y_pred_thresh))

# Plotar o gráfico de Métricas vs Threshold
plt.figure(figsize=(10, 6))
plt.plot(thresholds, accuracy_list, label='Acurácia', lw=2)
plt.plot(thresholds, precision_list, label='Precisão', lw=2)
plt.plot(thresholds, recall_list, label='Recall', lw=2)
plt.plot(thresholds, f1_list, label='F1 Score', lw=2)
plt.xlabel('Threshold')
plt.ylabel('Valor da Métrica')
plt.title('Métricas de Classificação vs Threshold')
plt.legend(loc='best')
plt.grid(True)
plt.show()

# Analisando modelo com o Threshold em 0.6

In [ ]:
# Definindo o threshold de 0.6
threshold = 0.60
y_prob = best_model.predict_proba(X_test)[:, 1]
y_pred_06 = (y_prob >= threshold).astype(int)

# Calculando as métricas para threshold = 0.6
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy_06 = accuracy_score(y_test, y_pred_06)
precision_06 = precision_score(y_test, y_pred_06)
recall_06 = recall_score(y_test, y_pred_06)
f1_06 = f1_score(y_test, y_pred_06)

print(f"Acurácia com threshold 0.6: {accuracy_06:.2f}")
print(f"Precisão com threshold 0.6: {precision_06:.2f}")
print(f"Recall com threshold 0.6: {recall_06:.2f}")
print(f"F1 Score com threshold 0.6: {f1_06:.2f}")


## Matriz de confusão com Threshold de 60%

In [ ]:
# Calcula a matriz de confusão
cm = confusion_matrix(y_test, y_pred_06)

# Define os rótulos customizados para as classes:
# 0 --> "não fechou o gap"
# 1 --> "fechou o gap"
classes = ['Não fechou o GAP', 'Fechou o GAP']

# Plota a matriz de confusão utilizando seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=classes, yticklabels=classes)
plt.xlabel('Valor Predito')
plt.ylabel('Valor Verdadeiro')
plt.title('Matriz de Confusão')
plt.show()

# Analisando os atributos

In [ ]:
# Nome colunas
feature_names = X.columns
# Fazer previsões e avaliar o modelo
y_pred = best_model.predict(X_test)
print("Acurácia do modelo:", accuracy_score(y_test, y_pred))

# Analisar a importância dos atributos
importances = best_model.feature_importances_

# Criar um DataFrame para exibir as importâncias de forma organizada
df_importances = pd.DataFrame({
    'Atributo': feature_names,
    'Importância': importances
}).sort_values(by='Importância', ascending=False)

print("\nImportância dos Atributos:")
print(df_importances)

df_importances = df_importances.reset_index(drop=True)

# Gerar um array com as posições (0, 1, 2, ..., n-1)
posicoes = np.arange(len(df_importances))

plt.figure(figsize=(16, 9))

# Plota as barras usando as posições numéricas
plt.bar(posicoes, df_importances['Importância'], color='skyblue', align='center')

# Define os rótulos (xticks) usando as mesmas posições
plt.xticks(posicoes, df_importances['Atributo'], rotation=45, ha='right')

plt.title('Importância dos Atributos - Random Forest')
plt.xlabel('Atributos')
plt.ylabel('Importância')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Exportando o modelo 

In [ ]:
import joblib
joblib.dump(best_model, 'melhor_modelo_prever_se_vai_fechar_o_gap.pkl')
print("Modelo exportado com sucesso!")